# Домашнее задание 1. Сбор данных

При решении реальных задач мы почти никогда не имеем дело с "хорошими" исходными данными, уже подготовленными для обработки и анализа. Как правило на входе имеются неструкткрированные данные в "грязном" виде, например необработанные тексты, изображения или аудио файлы. Иногда нет даже этого, и данные приходится собирать из разных доступных источников: разнообразных баз данных, внешних сервисов и даже электронных таблиц. После того, как данные получены, их унифицируют, очищают от шума, преобразовывают в нужный вид и сохраняют для дальнейшего анализа. В одном из традиционных подходов к Data Mining этот процесс называется Extract-Transform-Load

In [1]:
!pip install -r requirements.txt

  Running setup.py bdist_wheel for vk ... - \ done
  Stored in directory: /home/artyoms/.cache/pip/wheels/8d/e2/0c/056e125d54b4c9b2830ca6a7e6de00b36d416775da969abb94
Successfully built vk


In [26]:
import vk
import folium
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

Нам потребуется загрузить информацию о местах, в которых совершаются чекины, иначе говоря провести сбор данных (data collection). [Ссылка на ВК для разработчиков](https://vk.com/dev)

Для того, чтобы получить доступ к API прежде всего необходимо зарегистрироваться в VK в качестве разработчика и создать свое [приложение](https://vk.com/apps?act=manage). После создания приложения будет доступен набор ключей, которые мы будем использовать для аутентификации. Далее вам потребуется создать токен авторизации, чтобы получать доступ к API. Этот токен необходимо скопировать в соответствующие константы ниже. Подробнее о том, как работает аутентификация в VK API можно почитать [по ссылке](https://vk.com/dev/first_guide), хотя это нужно скорее для ознакомления: библиотека обращения с API позаботится о механизме аутентификации за нас.

In [35]:
ACCESS_TOKEN = "dbe01992331f7d49eb88950126aaf0b12b04be51e22cac325d329805c4d78fcdcc586961c0f8347819f5f"
session = vk.Session(access_token=token)
vkapi = vk.API(session=session)

In [65]:
print vkapi.getServerTime()
print vk.version


1489176640
2.0.2


VK API может отдавать вам посещаемые места несколькими способами: в некотором радиусе от точки или же по id самого места. Мы будем пользоваться [вторым способом](https://vk.com/dev/places.getById) . Требуется написать функцию, которая на вход получает объект подключения к API и индекс, до какого объекта требуется запросить. Возвращать она будет pandas.DataFrame с полученными данными.

In [141]:
def get_place_info(vkapi, idx=5000):
    current_idx = 1
    frames = []
    while(current_idx < idx):
        if current_idx + 1000 < idx:    
            places_id = range(current_idx,current_idx + 1001,1)
        else:
            places_id = range(current_idx,idx+1,1)
            
        current_idx +=1001
        places_list = vkapi.places.getById(places=places_id)
        frames.append(pd.DataFrame({"Title": [place['title'] for place in places_list], 
                         "Latitude":[place['latitude'] for place in places_list],
                         "Longitude":[place['longitude'] for place in places_list],
                         "Created":[place['created'] for place in places_list],
                         "Checkins":[place['checkins'] for place in places_list],
                         "Type":[place['type'] for place in places_list], 
                         "Country":[place['country'] for place in places_list], 
                         "City":[place['city'] for place in places_list], 
                         "Address":[place['address'] for place in places_list]},
                         index=[place['pid'] for place in places_list]))
        
    return pd.concat(frames)

Предположим, что мы хотим понять несколько фактов о выборке объектов с id от 1 до 5000.
Помогите нам это сделать.
А вот и сами факты:
1. Провизуализируйте на карте все объекты которые находятся в Москве? Где больше объектов, в центре или на окраине?
2. Посчитайте количество объектов разного типа. Постройте распределение? Объектов какого типа больше в данной выборке?
3. Посчитайте распределение числа чекинов по всей выборке. Визуализируйте его.
4. Постройте box plot по числу чекинов для каждого типа объектов.
5. Преобразуйте поле created в datetime. Как давно были созданы объекты? Сколько объектов существует больше года? 

Не забудьте сохранить данную выборку, в будущем она вам понадобится.

In [142]:
data = get_place_info(vkapi=vkapi, idx=5000)

In [143]:
print data

                          Address  Checkins   City  Country     Created  \
1               Мячковский б-р 3а       435      1        1  1280741941   
2                       Дачная 2а        14    123        1  1280741941   
3                Лени Голикова 29         8      2        1  1280741941   
4            Героев Танкограда 63         8    158        1  1280741941   
5               Мусы Джалиля 14К1        38     99        1  1280741941   
6                 Мусы Джалиля 14         6     99        1  1280741941   
7                       Труда 153        15    158        1  1280741941   
8                  Сони Кривой 81         3    158        1  1280741941   
9               просп. Ленина 66а       161    158        1  1280741941   
10                   Гурьянова 31        13      1        1  1280741941   
11                   Энгельса 1К1         1     99        1  1280741941   
12              Железноводская 68         3      2        1  1280741941   
13                  ул.Кр

In [11]:
a = [[1,2,3],[1,2,3],[1,2,3]]
print a[1][2]

3
